# Robo Advisor - Dashboard

## Content
* [1. Loading the Libraries and the data](#1)
* [2. Code for the dashboard Interface](#2)
* [3. Code for the underlying functions within the interface](#3)


#### Note that the dashboard opens up in a separate browser. The url for the browser will be produced in the end of the code and would look something like "http://127.0.0.1:8080"

<a id='1'></a>
## 1. Loading the Libraries and the data

Checking if the additional packages needed are present, if not install them. These are checked separately as they aren't included in requirement.txt as they aren't used for all case studies.

Importing the packages needed

In [1]:
import dash
#import dash_core_components as dcc
#import dash_html_components as html
import dash.dcc as dcc
import dash.html as html
from dash.dependencies import Input,Output,State
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import dash_daq as daq
from pickle import load
import cvxopt as opt
from cvxopt import blas, solvers
import sklearn

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage,risk_matrix

from pypfopt import plotting
import copy

### Load the data of the investors/individuals

In [2]:
# df.head()
investors = pd.read_csv('InputData.csv', index_col = 0 )
investors.head(1)

,AGE07,EDCL07,MARRIED07,KIDS07,LIFECL07,OCCAT107,INCOME07,RISK07,WSAVED07,SPENDMOR07,NETWORTH07,TrueRiskTol
0,47,2,1,0,2,1,56443.744181,3,1,5,352641.7113,6.947439


### Load the market data and clean the data

In [3]:
assets = pd.read_csv('../assets.csv',index_col=0)
#missing_fractions = assets.isnull().mean().sort_values(ascending=False)

#missing_fractions.head(10)

#drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))

#assets.drop(labels=drop_list, axis=1, inplace=True)
#assets.shape
# Fill the missing values with the last value available in the dataset. 
#assets=assets.fillna(method='ffill')
#assets.head(2)

In [4]:
assets.columns#.describe()

Index(['ITOT', 'IVV', 'QQQ', 'VTI', 'IJR', 'VPL', 'VWO', 'SUSA', 'HYEM',
       'MGV'],
      dtype='object')

In [5]:
options=np.array(assets.columns)
# str(options)
options = []

for tic in assets.columns:
    #{'label': 'user sees', 'value': 'script sees'}
    mydict = {}
    mydict['label'] = tic #Apple Co. AAPL
    mydict['value'] = tic
    options.append(mydict)

<a id='2'></a>
## 2. Code for the dashboard Interface

In [6]:
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [7]:
app.layout = html.Div([
    html.Div([ 
        #Dashboard Name
        html.Div([
            html.H3(children='Robo Advisor Dashboard'),
            html.Div([
                html.H5(children='Step 1 : Enter Investor Characteristics '),            
                ],style={'display': 'inline-block','vertical-align': 'top',  'width': '30%',\
                         'color':'black', 'background-color': 'LightGray'}), 
            html.Div([
                html.H5(children='Step 2 : Asset Allocation and portfolio performance'),            
                ],style={'display': 'inline-block', 'vertical-align': 'top',  \
                         'color':'white','horizontalAlign' : "left", 'width': '70%', 'background-color':'black'}), 
            ],style={'font-family': 'calibri'}),        
         
         #All the Investor Characteristics
                      
       #********************Demographics Features DropDown********
         html.Div([   
          html.Div([ 
            
            html.Label('Edad:',style={'padding': 5}),
            dcc.Slider(
                id='Edad',
                min = 18,
                max = 80,
                marks={ 25: '25',35: '35',45: '45',55: '55',65: '65'},              
                value=30),   
            #html.Br(),
            
            html.Label('Niv. Educ.:', style={'padding': 5}),
            dcc.Slider(
                id='neduc_ent',
                #min = investors['NETWORTH07'].min(), 
                min = 0.0, max = 5.0,
                marks={0.0: 'Sin educación',1.0: 'Ed. diferencial',2.0: "Ed. Básica",3.0: 'Ed. Media',4.0: 'CFT o IP',5.0 :"Universitaria",6.0:"Postgrado"},                
                value=3.0),
            #html.Br(),
            html.Label('Personas en el hogar:', style={'padding': 5}),
            dcc.Slider(
                id='tr_numh',
                #min = investors['INCOME07'].min(), max = investors['INCOME07'].max(),
                min = 1.0,
                max = 7.0,
                marks={1.0: 'entre 1 y 2',2.0: 'entre 3 y 4',3.0: 'entre 5 y 6',4.0: '+6 ',},
                value=2.0),
            
           # html.Br(),
            html.Label('Education Level (scale of 4):', style={'padding': 5}),
            dcc.Slider(
                id='Edu',
                min = investors['EDCL07'].min(), max = investors['EDCL07'].max(),
                marks={ 1: '1',2: '2',3: '3',4: '4'},
                value=2), 
            #html.Br(),
            html.Label('Married:', style={'padding': 5}),
            dcc.Slider(
                id='Married',
                min = investors['MARRIED07'].min(), max = investors['MARRIED07'].max(),
                marks={ 1: '1',2: '2'},
                value=1), 
            #html.Br(),
            html.Label('Kids:', style={'padding': 5}),
            dcc.Slider(
                id='Kids',
                min = investors['KIDS07'].min(), max = investors['KIDS07'].max(),
                #marks={ 1: '1',2: '2',3: '3',4: '4'},
                marks=[{'label': j, 'value': j} for j in investors['KIDS07'].unique()],
                value=3), 
            #html.Br(),
            html.Label('Occupation:', style={'padding': 5}),
            dcc.Slider(
                id='Occ',
                min = investors['OCCAT107'].min(), max = investors['OCCAT107'].max(),
                marks={ 1: '1',2: '2',3: '3',4: '4'},
                value=3),            
            #html.Br(),
            html.Label('Willingness to take Risk:', style={'padding': 5}),
            dcc.Slider(
                id='Risk',
                min = investors['RISK07'].min(), max = investors['RISK07'].max(),
                marks={ 1: '1',2: '2',3: '3',4: '4'},
                value=3), 
            #html.Br(),
            html.Button(id='investor_char_button',
                            n_clicks = 0,
                            children = 'Calculate Risk Tolerance',
                            style = {'fontSize': 14, 'marginLeft': '30px', 'color' : 'white',\
                                     'horizontal-align': 'left','backgroundColor': 'grey'}),             
            #html.Br(),            
              ],style={'width': '80%'}),           
            
            ],style={'width': '50%', 'font-family': 'calibri','vertical-align': 'top','display': 'inline-block'\
                     }),
#                     , "border":".5px black solid"}),

    # ********************Risk Tolerance Charts********            
       html.Div([    
               #html.H5(children='Step 2 : Enter the Instruments for the allocation portfolio'),  
        html.Div([
            html.Div([ 
                html.Label('Risk Tolerance (scale of 100) :', style={'padding': 5}),
                dcc.Input(id= 'risk-tolerance-text'),
               
                ],style={'width': '100%','font-family': 'calibri','vertical-align': 'top','display': 'inline-block'}),

            html.Div([ 
                html.Label('Select the assets for the portfolio:', style={'padding': 5}),
                dcc.Dropdown(
                        id='ticker_symbol',
                        options = options,
                        value = list(assets.columns),#['GOOGL', 'FB', 'GS','MS','GE','MSFT'], 
                        multi = True
                        # style={'fontSize': 24, 'width': 75}
                        ),
                html.Button(id='submit-asset_alloc_button',
                            n_clicks = 0,
                            children = 'Submit',
                            style = {'fontSize': 12, 'marginLeft': '25px','color' : 'white', 'backgroundColor': 'grey'}

                ), 
               ],style={'width': '100%','font-family': 'calibri','vertical-align': 'top','display': 'inline-block'}),
            ],style={'width': '100%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top'}),
           
           html.Div([                
                html.Div([
                    dcc.Graph(id='Asset-Allocation'), 
                    ], style={'width': '50%', 'vertical-align': 'top', 'display': 'inline-block', \
                      'font-family': 'calibri', 'horizontal-align': 'right'}),
                html.Div([
                    dcc.Graph(id='Performance')
                    ], style={'width': '50%', 'vertical-align': 'top', 'display': 'inline-block', \
                      'font-family': 'calibri', 'horizontal-align': 'right'}),
                   ], style={'width': '100%', 'vertical-align': 'top', 'display': 'inline-block', \
                          'font-family': 'calibri', 'horizontal-align': 'right'}),          


        ], style={'width': '70%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top', 'horizontal-align': 'right'}),
       ],style={'width': '70%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top'}),               

  ])    

<a id='3'></a>
## 3. Code for the underlying functions within the interface

The steps performed are as follows: 

1) Loading the regression model for predicting risk tolerance

2) Using markovitz mean variance analysis for asset allocation

3) Producing chart for the asset allocation and portfolio performance

#### Click the url produced by this code to see the dashboard

In [16]:
def predict_riskTolerance(X_input):

    filename = '../product/risk_model.p'
    loaded_model = load(open(filename, 'rb'))
    # estimate accuracy on validation set
    predictions = loaded_model.predict(X_input)
    print("\n\n")
    print(X_input)
    print(predictions)
    print("\n\n")
    return float(predictions)

#Asset allocation given the Return, variance
"""def get_asset_allocation(riskTolerance,stock_ticker):
    #ipdb.set_trace()
    assets_selected = assets.loc[:,stock_ticker]
    return_vec = np.array(assets_selected.pct_change().dropna(axis=0)).T
    n = len(return_vec)
    returns = np.asmatrix(return_vec)
    mus = 1-riskTolerance
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(return_vec))
    pbar = opt.matrix(np.mean(return_vec, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    w=portfolios['x'].T
    print (w)
    Alloc =  pd.DataFrame(data = np.array(portfolios['x']),index = assets_selected.columns)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    returns_final=(np.array(assets_selected) * np.array(w))
    returns_sum = np.sum(returns_final,axis =1)
    returns_sum_pd = pd.DataFrame(returns_sum, index = assets.index )
    returns_sum_pd = returns_sum_pd - returns_sum_pd.iloc[0,:] + 100   
    return Alloc,returns_sum_pd
"""
#Asset allocation given the Return, variance
def get_asset_allocation(riskTolerance,stock_ticker = assets.columns,initial_inv = 100):
    #ipdb.set_trace()
    assets_selected = assets.loc[:,stock_ticker]
    #return_vec = np.array(assets_selected.pct_change().dropna(axis=0)).T
    mu = mean_historical_return(assets,compounding = False)
    #S = CovarianceShrinkage(assets).ledoit_wolf()
    S = risk_matrix(assets)#.sample_cov()
    
    ef = EfficientFrontier(mu, S)
    print("ASSET")
    print(riskTolerance)
    print("\n")
    risk = np.sqrt(riskTolerance)

    weights = ef.efficient_risk(risk)
    ef.portfolio_performance(verbose=True)

    
    
    #mus = 1-riskTolerance
    
    # Convert to cvxopt matrices
    #S = opt.matrix(np.cov(return_vec))
    #pbar = opt.matrix(np.mean(return_vec, axis=1))
    
    # Create constraint matrices
    #G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    #h = opt.matrix(0.0, (n ,1))
    #A = opt.matrix(1.0, (1, n))
    #b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    #portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    #w=portfolios['x'].T
    #print (weights)
    #Alloc =  pd.DataFrame(data = np.array(portfolios['x']),index = assets_selected.columns)

    # Calculate efficient frontier weights using quadratic programming
    #portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    returns_final= np.dot(assets.loc[:, weights.keys()], np.array(list(weights.values())))
    #returns_sum = np.cumsum(returns_final)
    #print(returns_sum)
    returns_sum_pd = pd.DataFrame(returns_final, index = assets.index )
    returns_sum_pd = returns_sum_pd - returns_sum_pd.iloc[0,:] + initial_inv   
    return weights,returns_sum_pd

In [20]:
get_asset_allocation(0.01)

ASSET
0.01


Expected annual return: 5.8%
Annual volatility: 10.0%
Sharpe Ratio: 0.38


(OrderedDict([('ITOT', 7.015461e-10),
              ('IVV', 8.629366e-10),
              ('QQQ', 0.0679131148127694),
              ('VTI', 6.45733e-10),
              ('IJR', 3.002583e-10),
              ('VPL', 3.9388e-10),
              ('VWO', 1.435757e-10),
              ('SUSA', 1.4310513e-09),
              ('HYEM', 0.7086193404428072),
              ('MGV', 0.2234675402653682)]),
                                     0
 Date                                 
 2012-12-31 00:00:00-05:00  100.000000
 2013-01-02 00:00:00-05:00  100.340751
 2013-01-03 00:00:00-05:00  100.353704
 2013-01-04 00:00:00-05:00  100.429490
 2013-01-07 00:00:00-05:00  100.441923
 ...                               ...
 2022-12-23 00:00:00-05:00  132.030443
 2022-12-27 00:00:00-05:00  131.806216
 2022-12-28 00:00:00-05:00  131.307438
 2022-12-29 00:00:00-05:00  132.060298
 2022-12-30 00:00:00-05:00  131.996505
 
 [2519 rows x 1 columns])

In [ ]:
#Callback for the graph
#This function takes all the inputs and computes the cluster and the risk tolerance


@app.callback(
     [Output('risk-tolerance-text', 'value')],
    [Input('investor_char_button', 'n_clicks'),
    Input('Edad', 'value'),Input('edu', 'value'),
    Input('Est_civil', 'value'), Input('Ocup', 'value'),
    Input('estrat', 'value'),Input('tr_num', 'value'),
    Input('act_toth', 'value'),Input('rci_dt', 'value')])
#get the x and y axis details 

def update_risk_tolerance(n_clicks,Age,Nwcat,Inccl,Risk,Edu,Married,Kids,Occ):
      
    #ipdb.set_trace()
    
    RiskTolerance = 1
    if n_clicks != None:
        X_input = [[edad,edu,est_civil,ocup,estrat,tr_num, act_toth,rci_dt]]
        RiskTolerance= predict_riskTolerance(X_input)
    #print(RiskAversion)
    #Using linear regression to get the risk tolerance within the cluster.    
    return list([round(float(RiskTolerance*100),2)])

@app.callback([Output('Asset-Allocation', 'figure'),
              Output('Performance', 'figure')],
            [Input('submit-asset_alloc_button', 'n_clicks'),
            Input('risk-tolerance-text', 'value')], 
            [State('ticker_symbol', 'value')
            ])
def update_asset_allocationChart(n_clicks, risk_tolerance, stock_ticker):
    print(risk_tolerance)
    
    Allocated, InvestmentReturn = get_asset_allocation(risk_tolerance,stock_ticker)  
    
    return [{'data' : [go.Bar(
                        x=list(weights.keys()),
                        y=list(weights.values()),
                        marker=dict(color='C0'),
                    ),
                    ],
            'layout': {'title':" Asset allocation - Mean-Variance Allocation"}

       },
            {'data' : [go.Scatter(
                        x=InvestmentReturn.index,
                        y=InvestmentReturn.iloc[:,0],
                        name = 'OEE (%)',
                        marker=dict(color='red'),
                    ),
                    ],
            'layout': {'title':"Portfolio value of $100 investment"}

       }]

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2023 18:13:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jan/2023 18:13:05] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


None
ASSET
None


Exception on /_dash-update-component [POST]
AttributeError: 'NoneType' object has no attribute 'sqrt'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/dash.

127.0.0.1 - - [02/Jan/2023 18:13:05] "POST /_dash-update-component HTTP/1.1" 500 -


0
ASSET
0


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/_callback.py

...

127.0.0.1 - - [02/Jan/2023 18:13:13] "POST /_dash-update-component HTTP/1.1" 500 -


10
ASSET
10


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/_callback.

127.0.0.1 - - [02/Jan/2023 18:13:13] "POST /_dash-update-component HTTP/1.1" 500 -


10
ASSET
10


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/martin/anaconda3/envs/roboadvisor/lib/python3.10/site-packages/dash/_callback.

127.0.0.1 - - [02/Jan/2023 18:13:15] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
help(app.run_server)

## Sample Dashboard

<img src="RoboDashboard.png" width="1400" />